# Capstone project: <br>Battle of neighborhoods: Urban farming in Berlin

### Introduction / Business problem
A client from Berlin is aiming to set up urban farming facilities around the city to provide fresh vegetables and herbs to local venues. The business motivation is to maintain a fresh source of ingridients for local restaurants, reduce costs and environmental pollution due to logistics and promote a self-sustaining local community.
<br><br>

For this project, a couple of assumptions will be made:
- the client already did previous consultations on whether such a business is sustainable and profitable.
- the farms (facilities) are big and sophisticated enough to grow and provide plants for several venues (growing in layers with fully optimized conditions), but small enough to fit into an urban area e.g. 6 x 10 x 3 meters.
- the facilities will either be built into existing infrastracture (renting space around the town) or will be assembled separately (looking like cool high-tech ship containers). Once a suitable location is found, we assume there is enough renting space and the costs are also considered.
- the grown plants i.e. produced ingridients will be transported to the venues by an environmentally friendly form of transportation e.g. bicycles.
- each farm can supply 15-20 venues (depending on how often they are operating and how good they are visited) and cover an area with a radius of two kilometers.
- the client is ready to invest in 10 facilities for now.
<br><br>

The questions we want to address are:
- which neighborhoods / areas in Berlin are suitable to install the farming facilities
- how many farms have to be installed and where to cover the needs of each area (given the above assumptions)
- for each identified area, are there any similarities between venues that can be considered while planing the farming
<br><br>

Further things to consider:
- the assumption that one farm can supply 15-20 venues is of course totally arbitrary and just serves for the sake of having some things to deal with. In a real life case, depending on the number of venues one farm has to supply, the size of the facility could be set appropriately. Also, one would have to check the real-life technological capability of such a farm. This is beyond scope of this project. 

## Data
For this project, the following data will be used:
- List of neighborhoods in Berlin from Wikipedia and their respective geographical location
- Foursquare location data on the venues that provide food requiring some vegetables or herbs.
<br><br>

For this analysis, the city of Berlin will be divided according to the boroughs (Bezirk) and localities (Neighborhoods) from the Wikipedia article:
https://de.wikipedia.org/wiki/Liste_der_Bezirke_und_Ortsteile_Berlins

In [46]:
import numpy as np
import pandas as pd
import json
import folium
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import geocoder # import geocoder

In [ ]:
url = "https://de.wikipedia.org/wiki/Liste_der_Bezirke_und_Ortsteile_Berlins"

df = pd.read_html(url, header=0)[2]
df = df[["Ortsteil", "Bezirk"]]

# Getting the location for each locality
geolocator = Nominatim(user_agent="tor_explorer")
for index, row in df.iterrows():
    location = geolocator.geocode(row["Ortsteil"])
    df.at[index, "Long"] = location.longitude
    df.at[index, "Lat"] = location.latitude

In [110]:
df.head()

,Ortsteil,Bezirk,Long,Lat
0,Mitte,Mitte,13.402376,52.517690
1,Moabit,Mitte,13.342542,52.530102
2,Hansaviertel,Mitte,13.341872,52.519123
3,Tiergarten,Mitte,13.357260,52.509778
4,Wedding,Mitte,13.341970,52.550123


Example of Forsquare venues for the *Moabit* in Berlin

In [133]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

neighborhood_latitude = df.loc[1]["Lat"] # neighborhood latitude value
neighborhood_longitude =df.loc[1]["Long"] # neighborhood longitude value

neighborhood_name = "Moabit" # neighborhood name
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# make the GET request
results = requests.get(url).json()["response"]['groups'][0]['items']

venues_list=[]
# return only relevant information for each nearby venue
venues_list.append([(
    v['venue']['name'], 
    v['venue']['location']['lat'], 
    v['venue']['location']['lng'],  
    v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = [
              'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']

nearby_venues.columns = [c.replace(' ', '_') for c in nearby_venues.columns]

In [135]:
# Here I am filtering for all entries with Restaurant in the Venue Category
# A more extensive list of food providing venues will be considered in the analysis
nearby_venues = nearby_venues[nearby_venues.Venue_Category.str.contains('Restaurant')].reset_index(drop=True)
nearby_venues.head()

,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Delphin Fisch & Steak,52.532330,13.343766,Seafood Restaurant
1,Alimentari e Vini,52.527793,13.338373,Italian Restaurant
2,geh Veg,52.532178,13.342044,Vegetarian / Vegan Restaurant
3,Lichtblick,52.532392,13.341196,Restaurant
4,Habe die Ehre,52.528024,13.338907,Austrian Restaurant
